----

## NVidia GPU Simulator

Burton Rosenberg
30 May 2023

-----

This code simulates an Nvidia GPU. The class keeps dictionaries for memory objects (ndarrays) and kernels (functions), that are setup before a launch message is sent.

The launch is serial, of course, but on the GPU it is parallel. The kernels are run in order to test out the logic of the kernels. Success on the simulator should mean success of the actual device.

A difficulty is I wanted the kernels to be defined similar to the nvcc global functions. This means they make reference to variables that are lexicographically out of scope. In nvcc this is taken care of by device references on the host which are attached to actual reference on the device. Here I use a dictionary to map the name of the ndarry or the kernel to the array or kernel, with a bit of preamble for each kernel making the dictionary lookup.


In [1]:

class GPU:
    
    def __init__(self):
        self.mem = {}
        self.ker = {}
        
    def addKernel(self,name,kernel):
        self.ker[name] = kernel

    def addMemory(self,name,ndarray):
        self.mem[name] = ndarray

    def launch(self,name,n_threads,args):       
        for i in range(n_threads):
            ctx = (self.mem,self.ker,i)
            self.ker[name](ctx,args)


### The dot product

The dot product of two vectors is done in parallel with two kernels,

- A straight forward component-wise multiplication
- A sum of all elements using a log n depth tree

Arranging the sum, this method uses a folding approach. Intuitively this method divides the array in half, and moves the top half over the bottom half, aligning the elements. The sum then updates the values in the lower half.

The loop invariant is somthing the answer S is always the sum of the first i elements in the array. Start i at the length of the array and half it every interation, until i is one.

<pre>

initial array:

   +---+---+---+---+---+---+---+---+
   | 0   1   2   3   4   5   6   7 |
   +---+---+---+---+---+---+---+---+

fold in half and add

   +---+---+---+---+
   | 4   5   6   7 |
   +---+---+---+---+
   +---+---+---+---+
+  | 0   1   2   3 |
   +---+---+---+---+
====================
   +---+---+---+---+
   | 4   6   8  10 |
   +---+---+---+---+

fold in half and add

   +---+---+
   | 8  10 |
   +---+---+
   +---+---+
+  | 4   6 |
   +---+---+
====================
   +---+---+
   |12  16 |
   +---+---+
   
one last time

   +---+
   |12 |
   +---+
   +---+
+  |16 |
   +---+
====================
   +---+
   |28 |
   +---+

</pre>

#### Warp considerations

If we use the initial cells of the array, thread lauches are for consecutive thread ID's. We use all the threads in a warp (until we are under array size of 32).

#### Memory considerations

Each thread has exculsive access to the two memory cells it works with.

#### Synchronization

Using the default stream, enqueue a sequence of thread launchs according to
the having blocksize.

#### Efficiency.

This has log n phases, each phase using n/2 threads.

In [2]:
import numpy as np

def test_dot_product(k):
    
    def mult_array(ctx,args):
        (_m,_k,_tid) = ctx
        (a, b) = map((lambda r: _m[r]), args)

        a[_tid] *= b[_tid]

    def fold_array(ctx,args):
        (_m,_k,_tid), (a,k) = ctx, args
        a = _m[a]

        a[_tid] += a[_tid+k]

    n = 2**k

    gpu = GPU()
    gpu.addKernel('mult_array',mult_array)
    gpu.addKernel('fold_array',fold_array)

    a = (np.random.randint(0,n,n)).astype(float)
    b = (np.random.randint(0,n,n)).astype(float)

    gpu.addMemory('a',a)
    gpu.addMemory('b',b)
    
    print(f'a: {a}')
    print(f'b: {b}')
    d = a.dot(b)

    gpu.launch('mult_array',n,('a','b'))
    for i in range(k):
        gpu.launch('fold_array',n//(2**(i+1)),('a',n//(2**(i+1))))
    print(f'calculated: {a[0]}, actual: {d}')


test_dot_product(4)


a: [ 2.  0.  0. 10. 10.  5.  1.  7. 13.  2.  5.  2.  6. 14.  2.  3.]
b: [15. 11.  4.  8. 14. 14.  1.  9.  6.  2.  0. 11.  2.  3. 13.  8.]
calculated: 592.0, actual: 592.0


### The partial sum

The array is updated so a[i] contains the sum of the original values found in a[j] for all j less than or equal to i.

The loop invariant is that blocks of size 2^k, starting of indices mutliples of 2^k, are correctly the partial sum array of just that block. Initially k=0 is satisfied trivially, and finally the k' for which 2^k'==n is the problem solved.

The update from k to k+1 takes pairs of consecutive blocks of size 2^k and makes the loop invariant right for the combined block of size. 2^{k+1}. 

<pre>
Initial array:

   +---+---+---+---+---+---+---+---+
   | 1   2   1   3 | 2   3   2   1 |   L.I. true for 2^0 = 1
   +---+---+---+---+---+---+---+---+

....

   +---+---+---+---+---+---+---+---+
   | 1   3   4   7 | 2   5   7   8 |   L.I true for 2^2 =4 
   +---+---+---+---+---+---+---+---+
                 |   |   |   |   |
                 |   V   V   V   V
                 +-&gt;   shift up
                     |   |   |   |
                     V   V   V   V
   +---+---+---+---+---+---+---+---+
   | 1   3   4   7 | 9  12  14  15 |   L.I true for 2^3 =8
   +---+---+---+---+---+---+---+---+               
</pre>


#### Warp considerations

Active threads are in consecutive thread ID's, so the warps are fully untilized 
except before the block size is 32. The code below launches n threads, but n/2
threads are suffcient. 

#### Memory considerations

Each thread has exclusive access to one location in the array and shared read access
to another. Reads by multiple threads to the same location can be satisfied through
caching and braoadcast.

#### Synchronization

Using the default stream, enqueue a sequence of thread launchs according to
the having blocksize.

#### Efficiency.

This has log n phases, each phase using n/2 threads.

In [3]:
def test_partial_sum(k):
    
    # GPU KERNEL
    
    def raise_block(ctx,args):
        (_m,_k,_tid), (a,k) =  ctx, args
        a = _m[a]

        t = _tid>>k
        if (t%2)==1: 
            j = (t<<k)-1
            a[_tid] += a[j]           

    # test functionality
    
    def partial_sum_cpu(b):
        for i in range(1,len(b)):
            b[i] += b[i-1]
            
    def dist(a,b):
        p = 0
        for i in range(len(a)):
            p = abs(a[i]-b[i])
        return p
            
    # GPU simulation
    
    gpu = GPU()
    gpu.addKernel('raise_block',raise_block)
    
    n = 2**k
    a = (np.random.randint(0,n,n)).astype(float)
    gpu.addMemory('a',a)
    
    b = a.copy()
    partial_sum_cpu(b)
    print(f'a: {a}')

    for phase in range(k):
        gpu.launch('raise_block',n,('a',phase))
         
    print(f'a: {a}')
    print(f'error: {dist(a,b)}')
    
    
test_partial_sum(5)


a: [ 3. 25.  1. 24. 10. 27. 30. 29.  0. 19.  8. 27.  3. 11.  2. 27. 29. 12.
 18. 10. 22. 26. 10. 22.  0.  4. 30. 31.  3. 31. 10.  1.]
a: [  3.  28.  29.  53.  63.  90. 120. 149. 149. 168. 176. 203. 206. 217.
 219. 246. 275. 287. 305. 315. 337. 363. 373. 395. 395. 399. 429. 460.
 463. 494. 504. 505.]
error: 0.0
